### Stratified Random Sampler
E4E Acoustic Species ID Project Intro

Jeannie Kim

> Note: Final code is in `sampler.py`. This is messy and also not written in a function.

> I used this `ipynb` to step through the process easier and check the outputs as I went along.

In [1]:
# libraries
import numpy as np
import pandas as pd
import math
import random
import csv

In [2]:
# getting file; will change later to a function
DATA_PATH = "Peru_2019_AudioMoth_Data_Full.csv"

df = pd.read_csv(DATA_PATH, delimiter=",")
df.dropna(subset=['Duration', 'Comment'], inplace=True)   # drop all rows with NaN Duration (?)
df.head() # what does this file look like?

C:\Users\jeann\AppData\Local\Temp\ipykernel_25776\3990951082.py:4: DtypeWarning: Columns (1,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH, delimiter=",")


,AudioMothCode,AudioMothID,SourceFile,Directory,FileName,FileSize,Encoding,NumChannels,SampleRate,AvgBytesPerSec,BitsPerSample,StartDateTime,Duration,Error,Comment,Artist,FileCreateDate,FileType,FileTypeExtension,MIMEType
0,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_145649.WAV,GRABADOR-SDZG-AM-1,20190616_145649.WAV,1802240,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 14:56,2.346667,NaN,Recorded at 14:56:49 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:11-08:00,WAV,WAV,audio/x-wav
1,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_150000.WAV,GRABADOR-SDZG-AM-1,20190616_150000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:00,60.000469,NaN,Recorded at 15:00:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:11-08:00,WAV,WAV,audio/x-wav
2,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_151000.WAV,GRABADOR-SDZG-AM-1,20190616_151000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:10,60.000469,NaN,Recorded at 15:10:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:13-08:00,WAV,WAV,audio/x-wav
3,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_152000.WAV,GRABADOR-SDZG-AM-1,20190616_152000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:20,60.000469,NaN,Recorded at 15:20:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:15-08:00,WAV,WAV,audio/x-wav
4,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_153000.WAV,GRABADOR-SDZG-AM-1,20190616_153000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:30,60.000469,NaN,Recorded at 15:30:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:16-08:00,WAV,WAV,audio/x-wav


Step 1: Find successful Audiomoths
- Successful means:
    - not Audiomoth 21, 19, 8, or 28
    - they have at least one clip an hour for all 24 hours
      - that are (at least?) a minute long

> Weird note: "AM-#" and "WWF-#" (later in data)

In [3]:
# put the CSV into a dictionary grouped by the AudioMothCodes
amDict = {g:x for g,x in df.groupby('AudioMothCode', sort=False)}
print(amDict.keys())
print(amDict['AM-1'])

dict_keys(['AM-1', 'AM-10', 'AM-11', 'AM-12', 'AM-13', 'AM-14', 'AM-15', 'AM-16', 'AM-17', 'AM-18', 'AM-2', 'AM-20', 'AM-22', 'AM-23', 'AM-24', 'AM-25', 'AM-26', 'AM-27', 'AM-29', 'AM-3', 'AM-30', 'AM-4', 'AM-5', 'AM-6', 'AM-7', 'AM-8', 'AM-9'])
     AudioMothCode       AudioMothID                              SourceFile  \
0             AM-1  243B1F055B2BEAB8  GRABADOR-SDZG-AM-1/20190616_145649.WAV   
1             AM-1  243B1F055B2BEAB8  GRABADOR-SDZG-AM-1/20190616_150000.WAV   
2             AM-1  243B1F055B2BEAB8  GRABADOR-SDZG-AM-1/20190616_151000.WAV   
3             AM-1  243B1F055B2BEAB8  GRABADOR-SDZG-AM-1/20190616_152000.WAV   
4             AM-1  243B1F055B2BEAB8  GRABADOR-SDZG-AM-1/20190616_153000.WAV   
...            ...               ...                                     ...   
2838          AM-1  243B1F055B2BEAB8  GRABADOR-SDZG-AM-1/20190711_161000.WAV   
2842          AM-1  243B1F055B2BEAB8  GRABADOR-SDZG-AM-1/20190712_162000.WAV   
2844          AM-1  243B1F055B2BEA

In [5]:
badAms = {'AM-21', 'AM-19', 'AM-8', 'AM-28'}  # set of Audiomoths to ignore

# create dictionary of dictionaries
# first dictionary: keys = audiomoths, values = second dict
# second dict: keys = hour value (0-24), values = df of valid clips

finDict = dict()

for am in amDict.keys():    # for each audiomoth
  
  # skip Audiomoths we know are bad
  if am in badAms:
    continue

  # check if the audiomoth is good
  # they have at least one clip an hour for all 24 hours
    # that are (at least?) a minute long
  
  # dataframe corresponding to current Audiomoth
  currAmDf = amDict[am]
  
  hourDict = dict() # second dict : keys = hour value (0-24), values = df of valid clips

  for i, row in currAmDf.iterrows():
   # get hour and duration
    # new: from 'Comment' column*
    # comment example: "Recorded at 11:00:00 19/06/2022..."
    # *WWF Audiomoths don't have 'StartDateTime'

    comm = currAmDf['Comment'][i]
    time = str(comm).split()[2]
    hour = int(time.split(':')[0])

    duration = float(currAmDf['Duration'][i])
    
    # second thing: duration needs to be at least a minute
    if (duration >= 60):
      
      if hour not in hourDict.keys(): # new hour
        # add a new item to the dictionary
        hourDict[hour] = []

      hourDict[hour].append(row.values)
      

  # is there a valid clip for every hour?
  if len(hourDict) == 24:
    # add the am to the final dictionary
    finDict[am] = hourDict

In [6]:
print(len(finDict))
print(finDict["AM-1"][0])

25
[array(['AM-1', '243B1F055B2BEAB8',
       'GRABADOR-SDZG-AM-1/20190617_000000.WAV', 'GRABADOR-SDZG-AM-1',
       '20190617_000000.WAV', 46080360, 1.0, 1.0, 384000.0, 768000.0,
       16.0, '17.06.2019 00:00', 60.00046875, nan,
       'Recorded at 00:00:00 17/06/2019 (UTC) by AudioMoth 243B1F055B2BEAB8 at gain setting 2 while battery state was 4.6V.',
       'AudioMoth 243B1F055B2BEAB8', '2019:12:20 03:14:51-08:00', 'WAV',
       'WAV', 'audio/x-wav'], dtype=object), array(['AM-1', '243B1F055B2BEAB8',
       'GRABADOR-SDZG-AM-1/20190617_001000.WAV', 'GRABADOR-SDZG-AM-1',
       '20190617_001000.WAV', 46080360, 1.0, 1.0, 384000.0, 768000.0,
       16.0, '17.06.2019 00:10', 60.00046875, nan,
       'Recorded at 00:10:00 17/06/2019 (UTC) by AudioMoth 243B1F055B2BEAB8 at gain setting 2 while battery state was 4.6V.',
       'AudioMoth 243B1F055B2BEAB8', '2019:12:20 03:14:53-08:00', 'WAV',
       'WAV', 'audio/x-wav'], dtype=object), array(['AM-1', '243B1F055B2BEAB8',
       'GRABADOR-SD

Step 2: Do the sampling

Now we have a list of good audiomoths
- we could definitely optimize this lmao but ill do that later i guess
- so if the audiomoth is good and the duration is good, put that row in a new dictionary somehow
  - we have a dict grouped by audiomoths
    - we could use a dict grouped by Valid audiomoths and by time (???)
      - key is 0-24, value is df of valid clips

ok done now we do the damples

and write to csv



In [8]:
toWrite = []
# for each valid audiomoth
for am in finDict.keys():
  # for each hour
  for hour in finDict[am].keys():
    toWrite.append(random.choice(finDict[am][hour]))

# open csv
with open('samples.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(df.columns)
    write.writerows(toWrite)